# pytorch GPU 튜토리얼
## GPU가 사용 가능한지 확인

In [ ]:
import torch

torch.cuda.is_available() # 기본 CPU이기 때문에 실패

In [ ]:
import torch

# ... 메뉴로 이동해서 Accelerator - GPU T4 x2로 변경한다.
torch.cuda.is_available() 

기본적으로 생성된 변수는 모두 CPU에 존재한다.

In [ ]:
list0 = [1,2,3,4]
x = torch.tensor(list0)
x.is_cuda

In [ ]:
x = x.to('cuda')
x.is_cuda

다음 코드를 cuda로 실행할 수 있도록 수정해보자.

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import os 

# 트레이닝 데이터셋을 다운로드한다.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

# 테스트 데이터셋을 다운로드한다.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(784, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
        
    # 포워드 패스    
    def forward(self, x):
        return self.linear_relu_stack(x)



In [ ]:




def train_loop(dataloader, model, loss_fn, optimzer, device):
    size = len(dataloader.dataset)
    model.train() # 모델을 훈련 모드로 설정
    
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        pred = model(X) # 포워드 패스 수행
        loss = loss_fn(pred, y) # CE 연산
        
        optimzer.zero_grad() # 0 으로 초기화
        loss.backward() # 역전파하여 그래디언트 계산
        optimzer.step() # 연산된 그래디언트를 사용해 파라미터를 업데이트
        
        if batch % 100 == 0: # 매 100회차 마다 다음 내용 출력
            loss, current = loss.item(), batch * len(X)
            #print(f'loss: {loss}, [{current:>5d}/{size:>5d}]')

def test_loop(dataloader, model, loss_fn, device):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    model.eval() # 모델을 실행 모드로 설정
    
    with torch.no_grad(): # 그래디언트 연산 안함
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            
            pred = model(X) # 포워드 패스 수행
            test_loss += loss_fn(pred, y) # CE 연산
            correct += (pred.argmax(1) == y).type(torch.float).sum().item() # 결과 일치하는지 확인
    
    test_loss /= num_batches
    correct /= size
    print(f'Test Error: \n 정확도: {(100*correct):>0.1f}% 평균 Loss: {test_loss:>8f}\n')
    


In [ ]:
def run(device):
    #device = 'cuda:0' if torch.cuda.is_available() else 'cpu'   
    #device = 'cpu'
    print(f"사용할 장치: {device}")

    model = DNN().to(device)

    learning_rate = 1e-3
    batch_size = 64
    epochs = 10


    loss_fn = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    for t in range(epochs):
        print(f'Epoch {t+1}\n--------------------------------------')
        train_loop(train_dataloader, model, loss_fn, optimizer, device)
        test_loop(train_dataloader, model, loss_fn, device)
    print("Done!")

In [ ]:
%timeit run('cpu')

In [ ]:
%timeit run('cuda:0')